In [1]:
! wget https://raw.githubusercontent.com/philipperemy/name-dataset/master/names_dataset/first_names.all.txt

--2020-09-07 07:52:31--  https://raw.githubusercontent.com/philipperemy/name-dataset/master/names_dataset/first_names.all.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1289773 (1.2M) [text/plain]
Saving to: ‘first_names.all.txt.1’

first_names.all.txt 100%[===================>]   1.23M  --.-KB/s    in 0.1s    

2020-09-07 07:52:32 (12.3 MB/s) - ‘first_names.all.txt.1’ saved [1289773/1289773]



In [2]:
! pip install Unidecode

In [19]:
import torch
import torch.nn as nn
import string
import random
import sys
import unidecode
from torch.utils.tensorboard import SummaryWriter

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

all_chars = string.printable
n_chars = len(all_chars)
File = unidecode.unidecode(open('first_names.all.txt').read())

In [21]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(RNN,self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.embed = nn.Embedding(input_size, hidden_size)
    self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self,x ,hidden, cell):
    out = self.embed(x)
    out, (hidden,cell) = self.lstm(out.unsqueeze(1), (hidden,cell))
    out = self.fc(out.reshape(out.shape[0],-1))
    return out, (hidden,cell)
  
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
    cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
    return hidden ,cell

In [25]:
class Generator():

  def __init__(self):
    self.chunk_len = 250
    self.num_epochs = 5000
    self.batch_size = 1
    self.print_every = 50
    self.hidden_size = 256
    self.num_layers = 2
    self.lr = 0.003
  
  def char_tensor(self, string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
      tensor[c] = all_chars.index(string[c])
    return tensor
  
  def get_random_batch(self):
    start_idx = random.randint(0, len(File) - self.chunk_len)
    end_idx = start_idx + self.chunk_len + 1
    text_str = File[start_idx:end_idx]
    text_input = torch.zeros(self.batch_size, self.chunk_len)
    text_target = torch.zeros(self.batch_size, self.chunk_len)

    for i in range(self.batch_size):
      text_input[i,:] = self.char_tensor(text_str[:-1])
      text_target[i,:] = self.char_tensor(text_str[1:])

    return  text_input.long(), text_target.long()
  
  def generate(self, initial_str = 'A', predict_len = 100, temperature=0.85):
    hidden,cell = self.rnn.init_hidden(batch_size=self.batch_size)
    initial_input = self.char_tensor(initial_str)
    predicted = initial_str

    for p in range(len(initial_str)-1):
      _, (hidden,cell) = self.rnn(
          initial_input[p].view(1).to(device), hidden,cell
          )
    last_char = initial_input[-1]

    for p in range(predict_len):
      output,(hidden,cell) = self.rnn(last_char.view(1).to(device), hidden,cell)
      output_dist = output.data.view(-1).div(temperature).exp()
      top_char = torch.multinomial(output_dist,1)[0]
      predicted_char = all_chars[top_char]
      predicted += predicted_char
      last_char = self.char_tensor(predicted_char)
    return predicted


  
  def train(self):
    self.rnn = RNN(n_chars, self.hidden_size, self.num_layers, n_chars).to(device)

    optimizer = torch.optim.Adam(self.rnn.parameters(), lr=self.lr)
    criterion = nn.CrossEntropyLoss()
    writer = SummaryWriter(f'runs/names0')
    print('=> Starting training')
    for epoch in range(1, self.num_epochs + 1):
      inp, target = self.get_random_batch()
      hidden ,cell = self.rnn.init_hidden(batch_size=self.batch_size)

      self.rnn.zero_grad()
      loss = 0
      inp = inp.to(device)
      target = target.to(device)

      for c in range(self.chunk_len):
        output, (hidden,cell) = self.rnn(inp[:,c], hidden,cell)
        loss += criterion(output, target[:,c])
      
      loss.backward()
      optimizer.step()
      loss = loss.item() / self.chunk_len

      if epoch % self.print_every == 0:
        print(f'Loss: {loss}')
        print(self.generate())
      writer.add_scalar('Training loss:', loss, global_step=epoch)


In [26]:
gennames = Generator()
gennames.train()

=> Starting training
Loss: 2.895451904296875
Akyaso

o

ver-anaensba
tkcvjaunyoa
vaibam-
myraluiutpabsn
kvyhars
sotatdv
abvuao

irla
ivma
re
usovl
Loss: 2.34737255859375
Ag<re
sutirrae
frajusa
attoo
vsonl
tandyav
tisloan
aoavhyna
qiatan
dossistran
deelame
trmsata
anari	a
Loss: 2.50009521484375
A) n
bishae
cacemsa
llerjyge
arhal
sehgey
kjhrie
pin
inwy
iarena
lbhryj
hateye
daete-boms
ie
hscary
v
Loss: 2.585453125
A)et
navine
teleig
szinnan-jortane
etledru
doaalie
adsvern
emia
citepen
handara
erenupos
lie
[erag-et
Loss: 2.217326416015625
Acylln
aphatke
gorinn
cryjare
eilla
sahie
hana
borlsys
corar
arona
jakeya
kaucurie
alda
eyan
kalavan

Loss: 2.80879443359375
A-tina
mahina
fushalyn
eleejae
yosthad
mary
nasesya
rwil
aechan
arrlalemarriza
dara
odenna
mosa
shfe
Loss: 2.503272216796875
A]init
sendhan
ykyl
shunsyn
duna-marki
dalinstenana
aze
sila
mibka
ghannah
tiledntrini
trifbord
frazs
Loss: 2.57166650390625
Arion
culneja
iamaed
dolen
tarias
henshan
darak
chandos
jardico
samarili
mhatiu
liy